In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
filtered_df = pd.read_csv('..\Res\gold.csv')

In [3]:
dates = filtered_df["Date"]

In [4]:
filtered_dates = set([date.replace('-', '') for date in dates])
'19950102' in filtered_dates

True

In [11]:
def year(date):
    return date[:4]

def month(date):
    return date[:6]

def isUSA(country):
    return 1.05 if 'USA' == country else 1.

def isUS(country):
    return 1.05 if 'US' == country else 1.

def f(x):
    d = {}
    d['AllNumMentions'] = x['AllNumMentions'].sum()
    d['Goldstein'] = x['goldsteinscale'].mean()
    d['MonthYear'] = x['monthyear'].mean()
    return pd.Series(d)

def to_base(cameo_code):
    if cameo_code.startswith('0'):
        cameo_code = cameo_code[1]
    else:
        cameo_code = cameo_code[:2]
    return cameo_code

sign = lambda x: (1, -1)[x < 0]

In [ ]:
def normalizations(df):
    norm_df = 

In [ ]:
full_df = pd.DataFrame()
for part in range(1, 27):
    df = pd.read_csv(f'..\Res\exports_all_part{part}.csv', 
                  sep='\t', 
                  names=['globaleventid', 'day', 'monthyear', 'actor1name', 'actor1countrycode', 'actor2name', 'actor2countrycode',
                        'eventcode', 'eventbasecode',
                        'eventrootcode', 'goldsteinscale', 'nummentions', 'numsources', 'avgtone', 'actor1geo_fullname',
                        'actor1geo_countrycode', 'actor2geo_fullname', 'actor2geo_countrycode', 'actiongeo_fullname',
                        'actiongeo_countrycode'],
                dtype={'globaleventid':str, 'day':str, 'monthyear':str, 'actor1name':str, 'actor1countrycode':str, 'actor2name':str, 'actor2countrycode':str,
                        'eventcode':str, 'eventbasecode':str,
                        'eventrootcode':str, 'goldsteinscale':float, 'nummentions':int, 'numsources':int, 'avgtone':float, 'actor1geo_fullname':str,
                        'actor1geo_countrycode':str, 'actor2geo_fullname':str, 'actor2geo_countrycode':str, 'actiongeo_fullname':str,
                        'actiongeo_countrycode':str})
    del df['globaleventid']
    del df['actor1name']
    del df['actor2name']
    del df['eventbasecode']
    del df['actor1geo_fullname']
    del df['actor2geo_fullname']
    del df['actiongeo_fullname']
    del df['actor2geo_countrycode']
    del df['actor1geo_countrycode']
    del df['avgtone']
    
    df['year'] = df['day'].apply(year).astype('int')
    df['monthyear'] = df['monthyear'].astype('int')
    df['day'] = df['day'].astype('int')

    df['AllNumMentions'] = (df['numsources'] + df['nummentions']) * df['actor1countrycode'].apply(isUSA) * df['actor2countrycode'].apply(isUSA) * df['actiongeo_countrycode'].apply(isUS)
    
    del df['actor1countrycode']
    del df['actor2countrycode']
    del df['actiongeo_countrycode']
    del df['eventrootcode']
    
    df = df.dropna()
    
    df = df.groupby(['day', 'eventcode']).apply(f).reset_index()
    
    df['BaseCode'] = df['eventcode'].apply(to_base)
    
    df['Coeff'] = df['AllNumMentions'] * np.power(1.3,abs(df['Goldstein'])) * df['Goldstein'].apply(sign)

    del df['eventcode']
    del df['AllNumMentions']
    del df['Goldstein']
    
    df = df.groupby(['day', 'BaseCode']).sum().reset_index()
    full_df = full_df.append(df)
    print(f'{part}. iteration ended.')
    part += 1 